In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/Hackerearth Challenges/Deeplearning 1')

In [3]:
pwd

'/content/drive/My Drive/Colab Notebooks/Hackerearth Challenges/Deeplearning 1'

In [4]:
import pandas as pd
import numpy as np
from keras.preprocessing import image
import matplotlib.pyplot as plt

In [5]:
# !wget -O data t.ly/Wh06

In [6]:
# !unzip data

In [7]:
df = pd.read_csv('./dataset/train.csv')
df.head()

,Image,Class
0,image3476.jpg,Miscellaneous
1,image5198.jpg,Candle
2,image4183.jpg,Snowman
3,image1806.jpg,Miscellaneous
4,image7831.jpg,Miscellaneous


In [8]:
df['Class'].value_counts()

Miscellaneous     2801
Christmas_Tree    1539
Jacket             640
Candle             593
Airplane           535
Snowman            361
Name: Class, dtype: int64

In [9]:
mapp = {}
rev_mapp = {}
curr = 0
for label in df['Class'].unique():
    mapp[label] = curr
    rev_mapp[curr] = label
    curr += 1

In [10]:
temp_data = {}
for index, row in df.iterrows():
    temp_data[row['Image']] = row['Class']

In [11]:
# temp_data

In [12]:
os.listdir()

['weights.h5',
 'data',
 'Transfer learning.ipynb',
 'BasicCNN.ipynb',
 'out.csv',
 'Resnet50.ipynb',
 'dataset']

## Training set

In [13]:
ih = 200
iw = 180

In [14]:
path = './dataset/train'
X_train = []
y_train = []

In [15]:
from tqdm import tqdm
for file in tqdm(os.listdir(path)):
    img = image.load_img(os.path.join(path, file), target_size = (ih,iw))
    x = image.img_to_array(img)
    X_train.append(x)
    y_train.append(mapp[temp_data[file]])

100%|██████████| 6469/6469 [00:10<00:00, 631.09it/s]


In [16]:
X_train = np.stack(X_train)
y_train = np.stack(y_train)

In [17]:
print(X_train.shape, y_train.shape)

(6469, 200, 180, 3) (6469,)


In [18]:
from sklearn.model_selection import train_test_split
XX_train, XX_test, yy_train, yy_test = train_test_split(X_train, y_train, test_size = 0.05)

In [19]:
print(XX_train.shape, XX_test.shape)

(6145, 200, 180, 3) (324, 200, 180, 3)


In [20]:
path = './dataset/test'
test_X = []
file_name = []

In [21]:
for file in tqdm(os.listdir(path)):
    file_name.append(file)
    img = image.load_img(os.path.join(path, file), target_size = (ih, iw))
    x = image.img_to_array(img)
    test_X.append(x)

100%|██████████| 3489/3489 [00:04<00:00, 767.09it/s]


In [22]:
test_X = np.stack(test_X)

In [23]:
test_X.shape

(3489, 200, 180, 3)

In [24]:
import tensorflow as tf

In [25]:
tf.keras.backend.clear_session()

In [26]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = {}):
        if(logs.get('acc') >= .999):
            print("Cancelling Training")
            self.model.stop_training = True

callbacks = myCallback()

In [27]:
from tensorflow.keras.applications import ResNet50

In [28]:
base_model = ResNet50(input_shape=(ih, iw, 3), include_top=False, weights="imagenet")
for layer in base_model.layers:
    layer.trainable = False

In [46]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D

base_model = Sequential()
base_model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
base_model.add(Dense(6, activation='softmax'))

In [47]:
base_model.compile(optimizer = tf.keras.optimizers.SGD(lr = 0.0001), loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

In [72]:
# base_model.fit(XX_train, yy_train, epochs = 20, batch_size = 16, validation_data=(XX_test, yy_test), callbacks = [callbacks])
base_model.fit(X_train, y_train, epochs = 10, batch_size = 16, callbacks = [callbacks])

Epoch 1/10
405/405 [==============================] - 58s 142ms/step - loss: 0.0383 - acc: 0.9859
Epoch 2/10
405/405 [==============================] - 57s 141ms/step - loss: 0.0337 - acc: 0.9895
Epoch 3/10
405/405 [==============================] - 57s 141ms/step - loss: 0.0350 - acc: 0.9879
Epoch 4/10
405/405 [==============================] - 57s 141ms/step - loss: 0.0326 - acc: 0.9898
Epoch 5/10
405/405 [==============================] - 57s 141ms/step - loss: 0.0301 - acc: 0.9900
Epoch 6/10
405/405 [==============================] - 57s 141ms/step - loss: 0.0291 - acc: 0.9898
Epoch 7/10
405/405 [==============================] - 57s 141ms/step - loss: 0.0306 - acc: 0.9906
Epoch 8/10
405/405 [==============================] - 57s 141ms/step - loss: 0.0318 - acc: 0.9879
Epoch 9/10
405/405 [==============================] - 57s 141ms/step - loss: 0.0273 - acc: 0.9906
Epoch 10/10
405/405 [==============================] - 57s 141ms/step - loss: 0.0297 - acc: 0.9921


## Prediction

In [73]:
y_hat = base_model.predict(test_X)

In [74]:
test_y = []

In [75]:
for lis in y_hat:
    index = np.argmax(lis)
    test_y.append(rev_mapp[index])

In [76]:
print(test_y[:5])

['Jacket', 'Jacket', 'Jacket', 'Christmas_Tree', 'Miscellaneous']


In [77]:
res = {}
for i in range(test_X.shape[0]):
    res[file_name[i]] = test_y[i]

In [78]:
result = pd.DataFrame(list(res.items()), columns = ['Image', 'Class'])

In [79]:
result.head()

,Image,Class
0,image8132.jpg,Jacket
1,image1490.jpg,Jacket
2,image3111.jpg,Jacket
3,image664.jpg,Christmas_Tree
4,image5802.jpg,Miscellaneous


In [80]:
print(result.head())

           Image           Class
0  image8132.jpg          Jacket
1  image1490.jpg          Jacket
2  image3111.jpg          Jacket
3   image664.jpg  Christmas_Tree
4  image5802.jpg   Miscellaneous


In [81]:
result.to_csv('out.csv',index = False)

In [82]:
dff = pd.read_csv('out.csv')

In [83]:
dff.head()

,Image,Class
0,image8132.jpg,Jacket
1,image1490.jpg,Jacket
2,image3111.jpg,Jacket
3,image664.jpg,Christmas_Tree
4,image5802.jpg,Miscellaneous
